In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [22]:
def check_df(df) :
    # print(df.head())
    for i, col_name in enumerate(df) :
        print(i, col_name, df[col_name].dtype)
        if df[col_name].hasnans :
            print(df[col_name].isna().value_counts())
        print('')
        if len(df[col_name].unique()) < 10 :
            print(df[col_name].value_counts())
        print('')

#### data.go.kr : 전국자동차정비업체표준데이터

- 지자치별 (광역시도) 자동차 정비업체 개수
- 지자치별 (광역시도) 자동차 정비업체, 영업상태별 개수 (영업상태를 column의 multi-index로 표현 / 01:영업, 02:휴업, 03:폐업)

In [23]:
pd_data = pd.read_csv('전국자동차정비업체표준데이터.csv', encoding='cp949')

pd_data.head()

for i,n in enumerate(pd_data.columns):
    print(i, n)

0 자동차정비업체명
1 자동차정비업체종류
2 소재지도로명주소
3 소재지지번주소
4 위도
5 경도
6 사업등록일자
7 면적
8 영업상태
9 폐업일자
10 휴업시작일자
11 휴업종료일자
12 운영시작시각
13 운영종료시각
14 전화번호
15 관리기관명
16 관리기관전화번호
17 데이터기준일자
18 제공기관코드
19 제공기관명


C:\Users\user\AppData\Local\Temp\ipykernel_124164\2330528487.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  pd_data = pd.read_csv('전국자동차정비업체표준데이터.csv', encoding='cp949')


In [24]:
check_df(pd_data)

0 자동차정비업체명 object


1 자동차정비업체종류 int64

3     40108
1      6613
2      2937
4       289
99       53
Name: 자동차정비업체종류, dtype: int64

2 소재지도로명주소 object
False    48972
True      1028
Name: 소재지도로명주소, dtype: int64


3 소재지지번주소 object
True     30324
False    19676
Name: 소재지지번주소, dtype: int64


4 위도 float64
False    49996
True         4
Name: 위도, dtype: int64


5 경도 float64
False    49996
True         4
Name: 경도, dtype: int64


6 사업등록일자 object


7 면적 object


8 영업상태 int64

1    47845
3     1997
2      158
Name: 영업상태, dtype: int64

9 폐업일자 object
True     48439
False     1561
Name: 폐업일자, dtype: int64


10 휴업시작일자 object
True     49870
False      130
Name: 휴업시작일자, dtype: int64


11 휴업종료일자 object
True     49986
False       14
Name: 휴업종료일자, dtype: int64


12 운영시작시각 object
False    25186
True     24814
Name: 운영시작시각, dtype: int64


13 운영종료시각 object
False    25185
True     24815
Name: 운영종료시각, dtype: int64


14 전화번호 object
False    36334
True     13666
Name: 전화번호, dtype: int64


15 관리기관명 object


16 관리기관전

In [25]:
sub_df = pd_data[['소재지도로명주소','소재지지번주소', '영업상태']]

filter1 = sub_df['소재지도로명주소'].isna()

sub_df['소재지도로명주소'][filter1] = sub_df['소재지지번주소'][filter1]

# print(sub_df['소재지도로명주소'].hasnans)

sub_df.drop('소재지지번주소', axis=1, inplace=True)

# print(sub_df.head())

sub_df['area_name'] = sub_df['소재지도로명주소'].apply(lambda x : x.split()[0])

# print(sub_df.head())

g1 = sub_df.groupby(['area_name', '영업상태'])
df1 = g1.count()

C:\Users\user\AppData\Local\Temp\ipykernel_124164\2597295949.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['소재지도로명주소'][filter1] = sub_df['소재지지번주소'][filter1]
C:\Users\user\AppData\Local\Temp\ipykernel_124164\2597295949.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df.drop('소재지지번주소', axis=1, inplace=True)
C:\Users\user\AppData\Local\Temp\ipykernel_124164\2597295949.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stabl

In [26]:
# df1.unstack(level=0)
df2 = df1.unstack()   # = df1.unstack(level=1)

df2.columns = ['영업', '휴업', '폐업']

# df2.fillna(0.0)
df3 = df2.replace(np.NaN, 0.0)

In [27]:
# csv 파일로 저장하는 방법
# df3.to_csv('지자체별_영업상태.csv', encoding='cp949')

import pickle

with open('지자체별_영업상태_pickle', 'wb') as f:
  pickle.dump(df3, f)

In [28]:
with open('지자체별_영업상태_pickle', 'rb') as f :
  df4 = pickle.load(f)

df4

,영업,휴업,폐업
area_name,,,
강원도,2491.0,8.0,28.0
경기도,12510.0,49.0,636.0
경상남도,2500.0,4.0,87.0
경상북도,3624.0,8.0,147.0
광주광역시,1240.0,1.0,190.0
대구광역시,2512.0,6.0,0.0
대전광역시,2132.0,11.0,292.0
부산광역시,1450.0,6.0,38.0
서울특별시,5479.0,20.0,68.0


In [29]:
df4 = df4.reset_index()

# df4.dtypes

df4['area_name'] = df4['area_name'].astype('category')



In [30]:
df4['area_cat_code'] = df4['area_name'].cat.codes

df4

,area_name,영업,휴업,폐업,area_cat_code
0,강원도,2491.0,8.0,28.0,0
1,경기도,12510.0,49.0,636.0,1
2,경상남도,2500.0,4.0,87.0,2
3,경상북도,3624.0,8.0,147.0,3
4,광주광역시,1240.0,1.0,190.0,4
5,대구광역시,2512.0,6.0,0.0,5
6,대전광역시,2132.0,11.0,292.0,6
7,부산광역시,1450.0,6.0,38.0,7
8,서울특별시,5479.0,20.0,68.0,8
9,세종특별자치시,226.0,1.0,0.0,9


In [31]:
list(df4['area_name'])

area_name_list = ['서울특별시',
                  '광주광역시',
                  '대구광역시',
                  '대전광역시',
                  '부산광역시',
                  '울산광역시',
                  '인천광역시',
                  '세종특별자치시',
                  '제주특별자치도',
                  '강원도',
                  '경기도',
                  '경상남도',
                  '경상북도',
                  '전라남도',
                  '전라북도',
                  '충청남도',
                  '충청북도']

In [32]:
from pandas.api.types import CategoricalDtype

area_cat = CategoricalDtype(categories = area_name_list, ordered=True)
df4['area_cat2'] = df4['area_name'].astype(area_cat)

print(df4.dtypes)
df4
df4['area_cat2_code'] = df4['area_cat2'].cat.codes # sklearn labelencoder

area_name        category
영업                float64
휴업                float64
폐업                float64
area_cat_code        int8
area_cat2        category
dtype: object


In [33]:
df4.sort_values('area_cat2_code')

,area_name,영업,휴업,폐업,area_cat_code,area_cat2,area_cat2_code
8,서울특별시,5479.0,20.0,68.0,8,서울특별시,0
4,광주광역시,1240.0,1.0,190.0,4,광주광역시,1
5,대구광역시,2512.0,6.0,0.0,5,대구광역시,2
6,대전광역시,2132.0,11.0,292.0,6,대전광역시,3
7,부산광역시,1450.0,6.0,38.0,7,부산광역시,4
10,울산광역시,917.0,0.0,130.0,10,울산광역시,5
11,인천광역시,2626.0,3.0,0.0,11,인천광역시,6
9,세종특별자치시,226.0,1.0,0.0,9,세종특별자치시,7
14,제주특별자치도,211.0,1.0,0.0,14,제주특별자치도,8
0,강원도,2491.0,8.0,28.0,0,강원도,9


In [34]:
sub_df = pd_data[['사업등록일자', '면적']]
# print(sub_df)

sub_df['면적'] = sub_df['면적'].replace('2351.3.', '2351.3')

sub_df['면적'] = sub_df['면적'].astype(np.float64)

sub_df.dtypes

C:\Users\user\AppData\Local\Temp\ipykernel_124164\1760749401.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['면적'] = sub_df['면적'].replace('2351.3.', '2351.3')
C:\Users\user\AppData\Local\Temp\ipykernel_124164\1760749401.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['면적'] = sub_df['면적'].astype(np.float64)


사업등록일자     object
면적        float64
dtype: object

In [35]:
# 년, 월, 일 구분
# sub_df['사업등록일자'].apply(lambda x: x.split('-'))

sub_df['사업등록일자'] = sub_df['사업등록일자'].astype('datetime64')
sub_df.dtypes

C:\Users\user\AppData\Local\Temp\ipykernel_124164\3893564580.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['사업등록일자'] = sub_df['사업등록일자'].astype('datetime64')


사업등록일자    datetime64[ns]
면적               float64
dtype: object

In [36]:
# sub_df.head()

sub_df['년'] = sub_df['사업등록일자'].dt.year
sub_df['월'] = sub_df['사업등록일자'].dt.month
sub_df['일'] = sub_df['사업등록일자'].dt.day

C:\Users\user\AppData\Local\Temp\ipykernel_124164\3633565831.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['년'] = sub_df['사업등록일자'].dt.year
C:\Users\user\AppData\Local\Temp\ipykernel_124164\3633565831.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df['월'] = sub_df['사업등록일자'].dt.month
C:\Users\user\AppData\Local\Temp\ipykernel_124164\3633565831.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

In [37]:
sub_df

,사업등록일자,면적,년,월,일
0,2006-11-27,650.00,2006,11,27
1,2016-02-17,8821.34,2016,2,17
2,2015-06-29,3837.80,2015,6,29
3,2000-02-07,246.00,2000,2,7
4,1998-07-13,516.00,1998,7,13
...,...,...,...,...,...
49995,2000-03-06,353.60,2000,3,6
49996,2006-07-06,633.10,2006,7,6
49997,2013-07-01,1516.77,2013,7,1
49998,1999-03-20,162.70,1999,3,20


In [38]:
sub_df['면적'].describe()

count    5.000000e+04
mean     9.655124e+02
std      1.949756e+04
min      0.000000e+00
25%      1.221675e+02
50%      2.448200e+02
75%      6.500000e+02
max      3.572654e+06
Name: 면적, dtype: float64

In [39]:
print(sub_df['면적'].quantile(0.5))
print(sub_df['면적'].quantile([0.25, 0.5, 0.75]))
print(sub_df['면적'].median())

244.82
0.25    122.1675
0.50    244.8200
0.75    650.0000
Name: 면적, dtype: float64
244.82


In [40]:
qv = sub_df['면적'].quantile([0.25, 0.75])
qv = qv.to_numpy()
print(qv)
iqr = qv[1] - qv[0]
print(iqr)

upper = qv[1] + 1.5*iqr
lower = qv[0] - 1.5*iqr
print(upper)
print(lower)

# 이상치 : upper 보다 큰것, lower 보다 작은 것
sub_df['면적']

filter1 = sub_df['면적']<lower
sub_df['면적'][filter1]

filter2 = sub_df['면적']>upper
sub_df['면적'][filter2]

filter2 = sub_df['면적']>upper
sub_df.loc[filter2, :]

[122.1675 650.    ]
527.8325
1441.74875
-669.58125


,사업등록일자,면적,년,월,일
1,2016-02-17,8821.34,2016,2,17
2,2015-06-29,3837.80,2015,6,29
9,2013-02-19,2604.46,2013,2,19
25,2014-06-19,2693.29,2014,6,19
26,2015-12-04,3280.00,2015,12,4
...,...,...,...,...,...
49975,2017-01-26,2990.40,2017,1,26
49983,2006-12-21,2790.70,2006,12,21
49988,2005-04-29,5819.70,2005,4,29
49992,2010-02-18,1806.30,2010,2,18
